In [1]:
import os
import xml.dom.minidom
import xml.etree.ElementTree as ET
from PIL import Image

### verify the integrity of images for test

In [ ]:
import os
import shutil
from PIL import Image

test_path = "../../yolo_hsil_s_20181128/test"
wrong_path = "../../yolo_hsil_s_20181128/wrong"
os.makedirs(wrong_path, exist_ok=True)

img_names = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".jpg")]
wrong = 0
for img_name in img_names:
    try:
        img = Image.open(img_name)
        img.close()
    except:
        wrong += 1
        shutil.move(img_name, wrong_path)
#         print("image corrupted", img_name)
print("total: {}, wrong: {}".format(len(img_names), wrong))

### pad test images to target sizes (608x608)
#### image at its original size, usually much smaller, is not good for direct yolov3 prediction

In [ ]:
import os
from PIL import Image

def resize_img_with_padding(img_name, size, save_path):
    img_name_new = os.path.join(save_path, os.path.basename(img_name))
    txt_name_new = os.path.splitext(img_name_new)[0] + ".txt"
    
    img = Image.open(img_name)
    
    # pad and crop image
    x = -((size - img.size[0]) // 2)
    y = -((size - img.size[1]) // 2)
    img_croped = img.crop(
        (
            x,
            y,
            size + x,
            size + y
        )
    )
    img_croped.save(img_name_new)
        
    # write lables to txt
    with open(txt_name_new, 'w') as f:
        f.write("padx {} pady {}\n".format(x, y))
            
#     print("processed ", img_name)

In [ ]:
test_path = "../../yolo_hsil_s_20181128/test"
test608_path = "../../yolo_hsil_s_20181128/test608"
size = 608

os.makedirs(test608_path, exist_ok=True)
img_names = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".jpg")]
for img_name in img_names:
    resize_img_with_padding(img_name, size, save_path=test608_path)

### generate xmls from prediction, jpg is not padded to target size

In [ ]:
def collect_labels(result_txt, class_i):
    all_labels = {}
    with open(result_txt, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().rsplit(' ', 5)  # split from right, in case there is blank space in jpg name
            basename = tokens[0]  # without file extension
            det = float(tokens[1])
            xmin, ymin = int(float(tokens[2])), int(float(tokens[3]))
            xmax, ymax = int(float(tokens[4])), int(float(tokens[5]))
            if not basename in all_labels:
                all_labels[basename] = []
            all_labels[basename].append({"det":det, "xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax, "name":class_i})
    return all_labels


def generate_xml(basename, labels, data_path, save_path):
    jpg_name = os.path.join(data_path, basename+".jpg")
    with Image.open(jpg_name) as img:
        w, h = img.size
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "folder"
    ET.SubElement(root, "filename").text = basename + ".jpg"
    ET.SubElement(root, "path").text = "path"

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(root, "segmented").text = "0"

    need_write = False
    for label in labels:
        if label["det"] > 0.2:
            need_write = True
            object = ET.SubElement(root, "object")
            ET.SubElement(object, "name").text = label["name"]
            ET.SubElement(object, "pose").text = "Unspecified"
            ET.SubElement(object, "truncated").text = "0"
            ET.SubElement(object, "difficult").text = "0"
            bndbox = ET.SubElement(object, "bndbox")          
            ET.SubElement(bndbox, "xmin").text = str(label["xmin"])
            ET.SubElement(bndbox, "ymin").text = str(label["ymin"])
            ET.SubElement(bndbox, "xmax").text = str(label["xmax"])
            ET.SubElement(bndbox, "ymax").text = str(label["ymax"])

    if not need_write:
        return
    raw_string = ET.tostring(root, "utf-8")
    reparsed = xml.dom.minidom.parseString(raw_string)
    with open(os.path.join(save_path, basename + ".xml"), "w") as f:
        f.write(reparsed.toprettyxml(indent="\t"))


def main(result_txt, class_i, data_path, save_path):
    os.makedirs(save_path, exist_ok=True)
    all_labels = collect_labels(result_txt, class_i)
    for basename,labels in all_labels.items():
        generate_xml(basename, labels, data_path, save_path)
        print("generated", basename+".xml")

In [ ]:
result_txt = "../../darknet/results/comp4_det_test_HSIL_S.txt"
class_i = "HSIL_S"
data_path = "../../yolo_hsil_s_20181128/test"
save_path = "../../yolo_hsil_s_20181128/test_pred"

main(result_txt, class_i, data_path, save_path)

### generate xmls from prediction, jpg is padded to target size

In [2]:
def collect_labels(result_txt, class_i):
    all_labels = {}
    with open(result_txt, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().rsplit(' ', 5)  # split from right, in case there is blank space in jpg name
            basename = tokens[0]  # without file extension
            det = float(tokens[1])
            xmin, ymin = int(float(tokens[2])), int(float(tokens[3]))
            xmax, ymax = int(float(tokens[4])), int(float(tokens[5]))
            if not basename in all_labels:
                all_labels[basename] = []
            all_labels[basename].append({"det":det, "xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax, "name":class_i})
    return all_labels


def generate_xml_with_padding(basename, labels, orig_path, pad_path, save_path):
    jpg_name = os.path.join(orig_path, basename+".jpg")
    with Image.open(jpg_name) as img:
        w, h = img.size
    
    pad_name = os.path.join(pad_path, basename+".txt")
    with open(pad_name, 'r') as f:
        line = f.readline()
        tokens = line.strip().split()
        padx, pady = int(tokens[1]), int(tokens[3])
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "folder"
    ET.SubElement(root, "filename").text = basename + ".jpg"
    ET.SubElement(root, "path").text = "path"

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(root, "segmented").text = "0"

    need_write = False
    for label in labels:
        if label["det"] > 0.2:
            need_write = True
            object = ET.SubElement(root, "object")
            ET.SubElement(object, "name").text = label["name"]
            ET.SubElement(object, "pose").text = "Unspecified"
            ET.SubElement(object, "truncated").text = "0"
            ET.SubElement(object, "difficult").text = "0"
            bndbox = ET.SubElement(object, "bndbox")          
            ET.SubElement(bndbox, "xmin").text = str(label["xmin"]+padx)
            ET.SubElement(bndbox, "ymin").text = str(label["ymin"]+pady)
            ET.SubElement(bndbox, "xmax").text = str(label["xmax"]+padx)
            ET.SubElement(bndbox, "ymax").text = str(label["ymax"]+pady)

    if not need_write:
        return
    raw_string = ET.tostring(root, "utf-8")
    reparsed = xml.dom.minidom.parseString(raw_string)
    with open(os.path.join(save_path, basename + ".xml"), "w") as f:
        f.write(reparsed.toprettyxml(indent="\t"))


def main(result_txt, class_i, orig_path, pad_path, save_path):
    os.makedirs(save_path, exist_ok=True)
    all_labels = collect_labels(result_txt, class_i)
    for basename,labels in all_labels.items():
        generate_xml_with_padding(basename, labels, orig_path, pad_path, save_path)
        print("generated", basename+".xml")

In [3]:
result_txt = "../../darknet/results/comp4_det_test_HSIL_S.txt"
class_i = "HSIL_S"
orig_path = "../../yolo_hsil_s_20181128/test"
pad_path = "../../yolo_hsil_s_20181128/test608"
save_path = "../../yolo_hsil_s_20181128/test_pred"

main(result_txt, class_i, orig_path, pad_path, save_path)

generated 2017-09-19-18_25_13_x48768_y39851_w112_h152.xml
generated 2017-09-15-10_46_25_x32791_y45447_w118_h146.xml
generated 2017-09-15-11_01_58_x49039_y42646_w166_h92.xml
generated 2017-09-07-19_08_00_x23439_y6329_w114_h108.xml
generated 2017-09-19-19_04_49_x22630_y15033_w124_h126.xml
generated 2017-09-15-10_30_47_x28285_y50837_w124_h130.xml
generated 2017-09-19-19_04_49_x38055_y52671_w120_h120.xml
generated 2017-09-19-18_02_56_x19342_y17804_w120_h116.xml
generated 2017-09-15-10_50_45_x14323_y36148_w114_h116.xml
generated 2017-09-15-10_53_10_x22720_y24814_w118_h150.xml
generated 2017-09-15-10_57_44_x28575_y38532_w124_h246.xml
generated 2017-09-19-19_04_49_x28346_y27807_w138_h110.xml
generated 2017-09-19-18_54_48_x30193_y53049_w108_h100.xml
generated 2017-09-19-18_22_29_x36633_y24122_w152_h128.xml
generated 2017-09-19-18_52_06_x12935_y13518_w102_h114.xml
generated 2017-09-15-10_30_47_x26869_y29041_w94_h136.xml
generated 2017-09-15-09_53_43_x26888_y12925_w100_h108.xml
generated 2017-09

generated 2017-09-19-19_04_49_x38742_y44726_w130_h130.xml
generated 2017-09-19-18_02_56_x45437_y44031_w86_h78.xml
generated 2017-09-15-10_48_41_x6217_y26550_w178_h180.xml
generated 2017-09-19-18_52_06_x21061_y36412_w94_h106.xml
generated 2017-09-19-18_38_46_x50007_y46713_w96_h102.xml
generated 2017-09-15-09_53_43_x30609_y21005_w166_h134.xml
generated 2017-09-19-18_54_48_x47825_y24454_w94_h164.xml
generated 2017-09-19-18_02_56_x16466_y44273_w92_h92.xml
generated 2017-09-19-18_02_56_x51673_y40434_w96_h106.xml
generated 2017-09-15-11_01_58_x44027_y54738_w126_h114.xml
generated 2017-09-15-10_14_30_x20420_y35016_w122_h102.xml
generated 2017-09-19-18_54_48_x28917_y23139_w132_h122.xml
generated 2017-09-19-19_04_49_x26416_y27436_w112_h200.xml
generated 2017-09-07-19_08_00_x29752_y39652_w158_h126.xml
generated 2017-09-15-10_22_58_x13914_y35608_w146_h150.xml
generated 2017-09-07-19_08_00_x11775_y40882_w122_h122.xml
generated 2017-09-19-19_04_49_x37647_y28156_w136_h104.xml
generated 2017-09-07-19

generated 2017-09-19-18_54_48_x31392_y24026_w126_h122.xml
generated 2017-09-15-10_46_25_x13018_y15075_w126_h122.xml
generated 2017-09-19-18_54_48_x34133_y53749_w118_h152.xml
generated 2017-09-15-09_53_43_x34670_y13325_w88_h122.xml
generated 2017-09-15-10_05_24_x48001_y15377_w134_h94.xml
generated 2017-09-19-18_08_49_x31935_y6951_w110_h118.xml
generated 2017-09-19-19_04_49_x18197_y41417_w116_h112.xml
generated 2017-09-15-09_53_43_x53829_y23259_w104_h78.xml
generated 2017-09-19-18_57_18_x13683_y10334_w112_h128.xml
generated 2017-09-19-17_59_45_x60499_y35292_w110_h140.xml
generated 2017-09-19-18_08_49_x28253_y27479_w106_h164.xml
generated 2017-09-15-10_57_44_x48082_y34884_w130_h98.xml
generated 2017-09-19-18_08_49_x22231_y27385_w136_h112.xml
generated 2017-09-19-18_08_49_x29839_y50770_w88_h130.xml
generated 2017-09-19-18_54_48_x51652_y15554_w120_h128.xml
generated 2017-09-15-10_14_30_x34981_y32631_w112_h112.xml
generated 2017-09-19-18_54_48_x50425_y25941_w130_h136.xml
generated 2017-09-19

generated 2017-09-15-10_57_44_x20911_y25225_w136_h122.xml
generated 2017-09-15-10_46_25_x48481_y42090_w98_h124.xml
generated 2017-09-15-10_30_47_x14229_y34824_w76_h156.xml
generated 2017-09-15-10_46_25_x27922_y38608_w142_h144.xml
generated 2017-09-15-10_35_04_x49191_y19696_w194_h160.xml
generated 2017-09-19-19_12_46_x38151_y27934_w138_h108.xml
generated 2017-09-15-09_53_43_x43941_y42633_w102_h98.xml
generated 2017-09-15-09_53_43_x47861_y12640_w102_h116.xml
generated 2017-09-15-10_14_30_x48257_y30061_w114_h96.xml
generated 2017-09-07-19_08_00_x26727_y45564_w102_h90.xml
generated 2017-09-19-18_28_21_x17422_y45945_w90_h98.xml
generated 2017-09-15-10_50_45_x23639_y22842_w136_h136.xml
generated 2017-09-19-18_02_56_x20819_y29627_w80_h102.xml
generated 2017-09-15-09_48_33_x19147_y20623_w130_h78.xml
generated 2017-09-19-19_04_49_x36358_y30377_w104_h140.xml
generated 2017-09-19-18_28_21_x46634_y14400_w106_h100.xml
generated 2017-09-15-10_30_47_x45039_y29148_w168_h130.xml
generated 2017-09-07-19

generated 2017-09-15-10_53_10_x26698_y21630_w116_h146.xml
generated 2017-09-07-19_42_51_x15913_y16504_w130_h188.xml
generated 2017-09-19-18_52_06_x15128_y44661_w118_h100.xml
generated 2017-09-19-18_52_06_x13872_y26816_w132_h132.xml
generated 2017-09-15-10_22_58_x37714_y9370_w132_h118.xml
generated 2017-09-19-19_04_49_x49776_y19165_w132_h136.xml
generated 2017-09-19-18_08_49_x49144_y31729_w98_h112.xml
generated 2017-09-15-10_53_10_x14497_y14722_w154_h156.xml
generated 2017-09-15-11_01_58_x28370_y3455_w104_h102.xml
generated 2017-09-15-10_14_30_x41256_y38901_w124_h114.xml
generated 2017-09-07-19_08_00_x25425_y43541_w126_h146.xml
generated 2017-09-15-10_14_30_x47800_y40063_w116_h122.xml
generated 2017-09-19-18_54_48_x25607_y23546_w164_h144.xml
generated 2017-09-07-19_08_00_x10112_y28206_w160_h128.xml
generated 2017-09-19-19_04_49_x33811_y52458_w170_h158.xml
generated 2017-09-07-19_08_00_x30903_y15410_w134_h122.xml
generated 2017-09-19-19_04_49_x19803_y26567_w128_h128.xml
generated 2017-09

...

generated 2017-09-19-19_04_49_x31904_y46535_w114_h148.xml
generated 2017-09-19-19_04_49_x14646_y25050_w180_h240.xml
generated 2017-09-15-10_30_47_x29253_y44969_w80_h120.xml
generated 2017-09-19-18_02_56_x23305_y25356_w110_h70.xml
generated 2017-09-19-18_54_48_x33016_y52927_w68_h130.xml
generated 2017-09-19-18_38_46_x57135_y25332_w130_h164.xml
generated 2017-09-19-18_57_18_x21278_y24190_w114_h122.xml
generated 2017-09-19-18_02_56_x40838_y28475_w100_h154.xml
generated 2017-09-07-19_08_00_x6146_y34817_w92_h96.xml
generated 2017-09-15-10_48_41_x20652_y42252_w124_h118.xml
generated 2017-09-19-18_52_06_x15613_y35347_w120_h108.xml
generated 2017-09-19-19_04_49_x8340_y14609_w124_h110.xml
generated 2017-09-07-19_08_00_x10843_y41990_w124_h116.xml
generated 2017-09-19-18_52_06_x25087_y7422_w138_h136.xml
generated 2017-09-15-10_05_24_x48629_y15716_w134_h126.xml
generated 2017-09-15-10_50_45_x27505_y11200_w114_h94.xml
generated 2017-09-15-10_50_45_x23659_y16582_w108_h126.xml
generated 2017-09-15-10

generated 2017-09-19-18_54_48_x39841_y28297_w176_h108.xml
generated 2017-09-15-10_57_44_x28916_y35629_w128_h162.xml
generated 2017-09-15-10_05_24_x33239_y20508_w108_h122.xml
generated 2017-09-15-10_18_28_x18252_y7942_w114_h102.xml
generated 2017-09-19-17_59_45_x19193_y23612_w110_h122.xml
generated 2017-09-19-18_25_13_x43694_y47404_w110_h184.xml
generated 2017-09-15-10_05_24_x26027_y31703_w146_h196.xml
generated 2017-09-15-10_20_33_x44076_y38138_w112_h92.xml
generated 2017-09-19-18_54_48_x25711_y41602_w116_h136.xml
generated 2017-09-15-10_53_10_x28374_y21622_w88_h188.xml
generated 2017-09-15-09_53_43_x35020_y56410_w238_h284.xml
generated 2017-09-19-18_54_48_x25751_y31075_w130_h140.xml
generated 2017-09-15-09_53_43_x44278_y27075_w130_h238.xml
generated 2017-09-15-10_50_45_x11072_y24280_w134_h124.xml
generated 2017-09-15-10_39_42_x34010_y8069_w116_h112.xml
generated 2017-09-15-10_53_10_x10123_y23078_w134_h154.xml
generated 2017-09-19-19_04_49_x29290_y50135_w106_h116.xml
generated 2017-09-

generated 2017-09-15-10_53_10_x17542_y24693_w118_h156.xml
generated 2017-09-19-18_28_21_x25898_y13417_w90_h108.xml
generated 2017-09-19-18_08_49_x12778_y16075_w142_h144.xml
generated 2017-09-19-18_52_06_x28744_y40478_w136_h136.xml
generated 2017-09-19-19_04_49_x24488_y23210_w96_h158.xml
generated 2017-09-15-10_35_04_x50902_y36995_w88_h172.xml
generated 2017-09-15-10_14_30_x44905_y41960_w86_h174.xml
generated 2017-09-19-19_04_49_x31258_y20454_w128_h230.xml
generated 2017-09-19-18_28_21_x10174_y28754_w120_h142.xml
generated 2017-09-19-19_04_49_x53113_y31212_w146_h136.xml
generated 2017-09-15-09_53_43_x16664_y31415_w100_h102.xml
generated 2017-09-07-19_42_51_x17738_y45241_w142_h122.xml
generated 2017-09-19-19_04_49_x12861_y43496_w152_h164.xml
generated 2017-09-19-18_38_46_x54736_y48107_w116_h130.xml
generated 2017-09-15-10_50_45_x12816_y20773_w150_h126.xml
generated 2017-09-19-18_02_56_x39741_y31700_w106_h74.xml
generated 2017-09-07-18_47_44_x15966_y39922_w202_h152.xml
generated 2017-09-1

generated 2017-09-15-10_39_42_x52854_y34941_w82_h144.xml
generated 2017-09-15-11_01_58_x26370_y50971_w116_h112.xml
generated 2017-09-19-19_04_49_x27243_y49276_w122_h106.xml
generated 2017-09-19-19_04_49_x11384_y21912_w128_h126.xml
generated 2017-09-19-18_02_56_x37280_y27069_w98_h130.xml
generated 2017-09-15-11_01_58_x20397_y49688_w100_h100.xml
generated 2017-09-15-10_03_15_x35453_y39516_w172_h94.xml
generated 2017-09-19-18_02_56_x56553_y28131_w78_h138.xml
generated 2017-09-07-19_08_00_x36556_y30372_w122_h146.xml
generated 2017-09-15-10_30_47_x15826_y46633_w124_h146.xml
generated 2017-09-15-10_14_30_x48654_y29306_w94_h88.xml
generated 2017-09-19-19_04_49_x28968_y45941_w130_h106.xml
generated 2017-09-19-18_02_56_x36480_y24417_w96_h86.xml
generated 2017-09-07-19_08_00_x9117_y46464_w246_h222.xml
generated 2017-09-19-18_41_52_x15785_y40830_w98_h130.xml
generated 2017-09-19-19_04_49_x26678_y44169_w120_h160.xml
generated 2017-09-15-10_50_45_x40216_y17847_w98_h98.xml
generated 2017-09-19-18_25

generated 2017-09-15-10_22_58_x17017_y17070_w106_h106.xml
generated 2017-09-07-19_08_00_x39062_y42578_w124_h108.xml
generated 2017-09-19-18_22_29_x13124_y32972_w114_h104.xml
generated 2017-09-19-18_25_13_x48896_y49223_w118_h116.xml
generated 2017-09-15-10_22_58_x36089_y28644_w134_h126.xml
generated 2017-09-19-18_08_49_x28871_y50307_w146_h142.xml
generated 2017-09-15-10_55_36_x18981_y41676_w102_h122.xml
generated 2017-09-15-10_03_15_x25646_y10412_w254_h92.xml
generated 2017-09-19-18_25_13_x34277_y26856_w142_h142.xml
generated 2017-09-15-10_53_10_x41037_y33089_w122_h126.xml
generated 2017-09-15-10_05_24_x56214_y33069_w144_h192.xml
generated 2017-09-19-19_04_49_x18370_y47714_w128_h118.xml
generated 2017-09-15-10_11_59_x29302_y6938_w142_h162.xml
generated 2017-09-15-10_11_59_x13099_y43651_w126_h140.xml
generated 2017-09-19-18_20_11_x21164_y25087_w110_h182.xml
generated 2017-09-15-10_22_58_x7078_y40359_w138_h150.xml
generated 2017-09-07-19_08_00_x26349_y5742_w122_h132.xml
generated 2017-09-